# 13442 words

In [56]:
import tensorflow as tf
import pickle
from tensorflow.keras import Sequential
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [57]:
with open('countvectorizer_2.pkl', 'rb') as f:
    count_vectorizer = pickle.load(f)

In [58]:
sentiment_mapping={
    '0':'sadness',
    '1':'joy',
    '2':'love',
    '3':'anger',
    '4':'fear',
    '5':['enthusiasm','surprise'],
}

In [59]:
X=['makes feel wonderful']

In [60]:
X=count_vectorizer.transform(X).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0]])

In [61]:
_,m=X.shape

In [62]:
opt=tf.keras.optimizers.Adam()
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
batch_size=1000

In [63]:
model=Sequential(
      [tf.keras.Input(shape=(m,)),
      tf.keras.layers.Dense(units=3000,activation='relu'),
      tf.keras.layers.Dense(units=600,activation='relu'),
      tf.keras.layers.Dense(units=400,activation='relu'),
      tf.keras.layers.Dense(units=260,activation='relu'),
      tf.keras.layers.Dense(units=30,activation='relu'),
      tf.keras.layers.Dense(units=6,activation='linear')])
    
model.compile(loss=loss,optimizer=opt,metrics='accuracy')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 3000)              40329000  
                                                                 
 dense_13 (Dense)            (None, 600)               1800600   
                                                                 
 dense_14 (Dense)            (None, 400)               240400    
                                                                 
 dense_15 (Dense)            (None, 260)               104260    
                                                                 
 dense_16 (Dense)            (None, 30)                7830      
                                                                 
 dense_17 (Dense)            (None, 8)                 248       
                                                                 
Total params: 42482338 (162.06 MB)
Trainable params: 4

In [64]:
model.load_weights('model_2.h5')

In [65]:
def clean_data(text):
    #removing @ tags 
    text=re.sub(r'@[a-zA-z0-9]+\s*',' ',str(text))
    
    #removing urls
    text=re.sub(r'http\S+', '', text)
    text=re.sub(r'www\.\S+', '', text)
    
    #replacing multiple whitesapces by a single
    text = re.sub(r'\s+',' ',text)
    
    # remove all single characters(surrounded by whitespace)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # remove all single characters except i and I (surrounded by whitespace)
    # text = re.sub(r'\s+(?![iI])[a-zA-Z]\s+', ' ', text)
    
    
    # Converting to Lowercase 
    text = text.lower()
    
    # Lemmatization- splits into list of words ['The', 'quick', ....]
    text = text.split()

    lemma = WordNetLemmatizer()
    text = [lemma.lemmatize(word) for word in text]
    text = ' '.join(text)
    
    words = nltk.word_tokenize(text)
    
    # Get the list of stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords from the text
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Recreate the text without stopwords
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

In [119]:
X=[
  "I am feeling so lonely",
]
X=[clean_data(x) for x in X]
print(X)
X=count_vectorizer.transform(X).toarray()
X

['feeling lonely']


array([[0, 0, 0, ..., 0, 0, 0]])

In [120]:
y=model.predict(X)
sentiment_mapping[str(np.argmax(y))]


1/1 [==============================] - 0s 104ms/step


'sadness'

In [118]:
y_sm=tf.nn.softmax(y)
y_sm

<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[5.7170931e-03, 1.4083543e-02, 2.2604350e-05, 9.5346838e-01,
        2.3734609e-02, 2.9734375e-03, 7.9450335e-10, 3.8292688e-07]],
      dtype=float32)>

In [26]:
pred=np.argmax(y_sm)
pred

1